In [247]:
#import data
import pandas as pd
emails = pd.read_csv("enron.csv", sep=";")
emails.head()

,id,subject,text,hamSpam
0,1,christmas tree farm pictures,NaN,1
1,2,vastar resources inc .,gary production from the high island larger ...,1
2,3,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,1
3,4,re : issue,fyi - see note below - already done . stella -...,1
4,5,meter 7268 nov allocation,fyi . - - - - - - - - - - - - - - - - - - - - ...,1


In [248]:
#drop unused data 
emails.drop(['id', 'subject'], axis="columns", inplace=True)
emails.head()

,text,hamSpam
0,NaN,1
1,gary production from the high island larger ...,1
2,- calpine daily gas nomination 1 . doc,1
3,fyi - see note below - already done . stella -...,1
4,fyi . - - - - - - - - - - - - - - - - - - - - ...,1


In [249]:
#1412 spam and 3663 ham emails
emails.groupby("hamSpam").describe()

text                                                               
        count unique                                                top freq
hamSpam                                                                     
0        1412   1353                          click here to be removed    19
1        3663   3415  > ricky a . archer fuel supply 700 louisiana  ...   21

In [250]:
#split data into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emails.text, emails.hamSpam, test_size=0.1)
print(len(y_test))

518


In [251]:
#transform text into numeric values
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values.astype('str'))
X_train_count.toarray()[:3]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [252]:
#create Multinomial Naive Bayes model, multinomial because we have discrete data (count of each word in text)
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count, y_train)

MultinomialNB()

In [253]:
#accuracy
#X_test_count = v.transform(X_test.values.astype('str'))
#model.score(X_test_count, y_test)

In [254]:
#function to get score
def get_score(model, X_train, X_test, y_train, y_test):
    v = CountVectorizer()
    X_train_count = v.fit_transform(X_train.values.astype('str'))
    X_test_count = v.transform(X_test.values.astype('str'))
    model.fit(X_train_count, y_train)
    return model.score(X_test_count, y_test)

In [255]:
# K-Folds cross-validator
#from sklearn.model_selection import KFold
#kf = KFold(n_splits=10)

In [256]:
#create 10 folds and their score precision recall and fscore
"""
scores = []
for train_index, test_index in kf.split(emails.text):
    X_train, X_test, y_train, y_test = emails.text[train_index], emails.text[test_index], \
                                        emails.hamSpam[train_index], emails.hamSpam[test_index]
    scores.append(get_score(model, X_train, X_test, y_train, y_test))
"""

'\nscores = []\nfor train_index, test_index in kf.split(emails.text):\n    X_train, X_test, y_train, y_test = emails.text[train_index], emails.text[test_index],                                         emails.hamSpam[train_index], emails.hamSpam[test_index]\n    scores.append(get_score(model, X_train, X_test, y_train, y_test))\n'

In [257]:
#Calculate precision_recall_fscore for test data (does same as the for loop above)
from sklearn.model_selection import cross_val_score
precision = cross_val_score(model, X_train_count, y_train, cv=10, scoring='precision')
recall = cross_val_score(model, X_train_count, y_train, cv=10, scoring='recall')
f1 = cross_val_score(model, X_train_count, y_train, cv=10, scoring='f1')
print("Precision: \n", precision)
print("Recall: \n", recall)
print("F1: \n", f1)

Precision: 
 [0.99384615 0.98784195 0.9875     0.99082569 0.9787234  0.99695122
 0.98784195 0.98484848 0.99386503 0.98466258]
Recall: 
 [0.97878788 0.98484848 0.95757576 0.98480243 0.9787234  0.99392097
 0.98784195 0.98784195 0.98480243 0.97568389]
F1: 
 [0.98625954 0.98634294 0.97230769 0.98780488 0.9787234  0.99543379
 0.98784195 0.98634294 0.98931298 0.98015267]


In [258]:
#Make predictions and calculate precision_recall_fscore for predictions
from sklearn.metrics import precision_recall_fscore_support
X_test_count = v.transform(X_test.values.astype('str'))
Y_pred = model.predict(X_test_count)
prf = precision_recall_fscore_support(Y_pred, y_test, average='macro')
prf

(0.9806951272754882, 0.9897688811526932, 0.9851140870164952, None)